In [31]:
import pandas as pd
import json
from scipy.optimize import minimize
import numpy as np

In [10]:
prev = pd.read_csv('data/TBPS_RU.csv')
prev = prev[prev.ClusterType == 'Urban']
prev = prev.set_index('Gp')
prev = {i: dict(row) for i, row in prev.iterrows()}
prev

{'Asym': {'ClusterType': 'Urban',
  'M': 0.631150299630622,
  'N': 205,
  'L': 0.5658536585365853,
  'U': 0.697560975609756},
 'ExCS': {'ClusterType': 'Urban',
  'M': 0.187873896500911,
  'N': 205,
  'L': 0.1365853658536585,
  'U': 0.2439024390243902},
 'PreCS': {'ClusterType': 'Urban',
  'M': 0.1329020199964461,
  'N': 205,
  'L': 0.0878048780487804,
  'U': 0.1804878048780487},
 'SpAsym': {'ClusterType': 'Urban',
  'M': 0.3290216415525827,
  'N': 139,
  'L': 0.2517985611510791,
  'U': 0.4100719424460431},
 'SpExCS': {'ClusterType': 'Urban',
  'M': 0.3790364467396788,
  'N': 30,
  'L': 0.2,
  'U': 0.5666666666666667},
 'SpPreCS': {'ClusterType': 'Urban',
  'M': 0.3892264961070054,
  'N': 36,
  'L': 0.2222222222222222,
  'U': 0.5555555555555556},
 'TB': {'ClusterType': 'Urban',
  'M': 0.0023535119209745,
  'N': 87617,
  'L': 0.002042982526222,
  'U': 0.0026821278975541}}

## Prevalence

In [27]:
def find_prev_slum(prev_all, tbor, pr_slum):
    pr = prev_all['M']

    def fn(x):
        pu, ps = x
        return ((ps / (1 - ps)) / (pu / (1 - pu)) - tbor) ** 2 + (pu * (1 - pr_slum) + ps * pr_slum - pr) ** 2


    sol = minimize(fn, [0.5, 0.5])
    pu, ps = sol.x
    
    m, l, u = prev_all['M'], prev_all['L'], prev_all['U']
    cu, cs = pu / m, ps / m
    
    return {
        'slum': {'Index': 'Prev', 'M': cs * m, 'L': cs * l, 'U': cs * u},
        'nonslum': {'Index': 'Prev', 'M': cu * m, 'L': cu * l, 'U': cu * u}
    }


prev_us = find_prev_slum(
    prev_all = prev['TB'],
    tbor = 2.26,
    pr_slum = 0.2
)

prev_us

{'slum': {'Index': 'Prev',
  'M': 0.004754368892159283,
  'L': 0.004127063255270464,
  'U': 0.0054182115362490894},
 'nonslum': {'Index': 'Prev',
  'M': 0.0021070667359413863,
  'L': 0.0018290540552390589,
  'U': 0.002401272083693806}}

## ARTI

In [39]:
def find_arti_slum(arti, rr, adr, pr_slum, year0, year1):
    m, l, u = arti['M'], arti['L'], arti['U']

    cu = 1 / (1 - pr_slum + pr_slum * rr) * np.exp(- adr * (year1 - year0))
    cs = cu * rr
    
    return {
        'slum': {'Index': 'ARTI', 'M': cs * m, 'L': cs * l, 'U': cs * u},
        'nonslum': {'Index': 'ARTI', 'M': cu * m, 'L': cu * l, 'U': cu * u}
    }


arti_us = find_arti_slum(
    arti = {'M': 0.022, 'L': 0.018, 'U': 0.026}, 
    rr = 1.24, 
    pr_slum = 0.2, 
    adr = 0.045,
    year0 = 2005, 
    year1 = 2020
)
arti_us

{'slum': {'Index': 'ARTI',
  'M': 0.013253613696730858,
  'L': 0.010843865751870702,
  'U': 0.015663361641591014},
 'nonslum': {'Index': 'ARTI',
  'M': 0.010688398142524886,
  'L': 0.008745053025702179,
  'U': 0.012631743259347593}}

In [40]:
prev_us, arti_us


({'slum': {'Index': 'Prev',
   'M': 0.004754368892159283,
   'L': 0.004127063255270464,
   'U': 0.0054182115362490894},
  'nonslum': {'Index': 'Prev',
   'M': 0.0021070667359413863,
   'L': 0.0018290540552390589,
   'U': 0.002401272083693806}},
 {'slum': {'Index': 'ARTI',
   'M': 0.013253613696730858,
   'L': 0.010843865751870702,
   'U': 0.015663361641591014},
  'nonslum': {'Index': 'ARTI',
   'M': 0.010688398142524886,
   'L': 0.008745053025702179,
   'U': 0.012631743259347593}})

In [56]:
for k, ent in prev.items():
    print(f"{k}\t{ent['M']:.0%} ({ent['L']:.0%}-{ent['U']:.0%})")

Asym	63% (57%-70%)
ExCS	19% (14%-24%)
PreCS	13% (9%-18%)
SpAsym	33% (25%-41%)
SpExCS	38% (20%-57%)
SpPreCS	39% (22%-56%)
TB	0% (0%-0%)


In [64]:
targets = [
    prev_us['slum'], 
    arti_us['slum'],
    {'Index': 'PrDR_CNR', 'M': 0.041, 'L': 0.035, 'U': 0.045}
]

for k in ['Asym', 'PreCS', 'ExCS', 'SpAsym', 'SpPreCS', 'SpExCS']:
    ent = prev[k]
    targets.append({
        'Index': f'Pr{k}',
        'M': ent['M'], 'L': ent['L'], 'U': ent['U']
    })


pd.DataFrame(targets)

,Index,M,L,U
0,Prev,0.004754,0.004127,0.005418
1,ARTI,0.013254,0.010844,0.015663
2,PrDR_CNR,0.041000,0.035000,0.045000
3,PrAsym,0.631150,0.565854,0.697561
4,PrPreCS,0.132902,0.087805,0.180488
5,PrExCS,0.187874,0.136585,0.243902
6,PrSpAsym,0.329022,0.251799,0.410072
7,PrSpPreCS,0.389226,0.222222,0.555556
8,PrSpExCS,0.379036,0.200000,0.566667


In [65]:
with open('data/targets.json', 'w') as f:
    json.dump(targets, f)